### Imports

In [1]:
# Define paths to the MAT files
edf_mat_path = "../data/EDF_RawData.mat"
xml_mat_path = "../data/XML_RawData.mat"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


from data.edf_data_import import (
    describe_edf_mat_structure,
    extract_eeg_signal,
    compute_signal_features,
    extract_all_eeg_signals,
    compute_psd,
)

## 1. EEG (Electroencephalogram) – Brain activity (2 channels)

In [4]:
# Display the results in a pandas DataFrame for better visualization.
all_eeg_signals = extract_all_eeg_signals(edf_mat_path, channel=0)
all_features = [compute_signal_features(signal) for signal in all_eeg_signals]
features_df = pd.DataFrame(all_features)
print("EEG Signal Features for All EDF Files:")
display(features_df)

EEG Signal Features for All EDF Files:


,mean,variance,rms,peak_to_peak,zero_crossings,hjorth_activity,hjorth_mobility,hjorth_complexity
0,16.394099,894.359849,34.104638,99.724193,4,894.359849,0.793813,1.756993
1,14.663747,990.666915,34.723081,126.187152,6,990.666915,0.924543,1.615575
2,14.801868,808.603764,32.057746,126.003281,4,808.603764,1.007303,1.371351
3,5.809894,1342.976985,37.104337,150.513085,6,1342.976985,0.937260,1.746578
4,4.229341,1201.752425,34.923341,158.348592,8,1201.752425,1.009617,1.491204
5,9.139617,1090.601269,34.265637,136.971466,8,1090.601269,0.799810,1.865842
6,15.861900,986.178284,35.182072,107.481594,6,986.178284,0.774596,1.838686
7,13.685445,1097.736304,35.847283,125.006104,6,1097.736304,0.716539,1.970163
8,23.262643,1460.344946,44.738076,132.352941,2,1460.344946,1.071323,1.479340
9,-5.475663,2002.536794,45.083475,174.241627,6,2002.536794,0.897918,1.619666


## 2. EOG (Electrooculogram) – Eye movements (2 channels: left and right eye)

In [ ]:
# Load the MAT file
# Calculate  Mean, variance, RMS, peak-to-peak, zero crossings

In [ ]:
#Frequency-domain features (FFT, Welch PSD):
#Relative power in delta, theta, alpha, beta, gamma bands
#	•	Bandpower ratios (e.g., delta/alpha, theta/beta)
#	•	Spectral entropy, median frequency

## 3. EMG (Electromyogram) – Muscle activity

In [ ]:
# Load the MAT file
# Calculate  Mean, variance, RMS, peak-to-peak, zero crossings

In [ ]:
#Frequency-domain features (FFT, Welch PSD):
#Relative power in delta, theta, alpha, beta, gamma bands
#	•	Bandpower ratios (e.g., delta/alpha, theta/beta)
#	•	Spectral entropy, median frequency

## 4. ECG (Electrocardiogram) – Heart activity

In [ ]:
# Load the MAT file
# Calculate  Mean, variance, RMS, peak-to-peak, zero crossings

In [ ]:
#Frequency-domain features (FFT, Welch PSD):
#Relative power in delta, theta, alpha, beta, gamma bands
#	•	Bandpower ratios (e.g., delta/alpha, theta/beta)
#	•	Spectral entropy, median frequency

## 5. Body Position

In [5]:
# Load the MAT file
# Calculate  Mean, variance, RMS, peak-to-peak, zero crossings

In [ ]:
#Frequency-domain features (FFT, Welch PSD):
#Relative power in delta, theta, alpha, beta, gamma bands
#	•	Bandpower ratios (e.g., delta/alpha, theta/beta)
#	•	Spectral entropy, median frequency

## 6. SpO2 (Oxygen Saturation)

In [ ]:
# Load the MAT file
# Calculate  Mean, variance, RMS, peak-to-peak, zero crossings

In [ ]:
#Frequency-domain features (FFT, Welch PSD):
#Relative power in delta, theta, alpha, beta, gamma bands
#	•	Bandpower ratios (e.g., delta/alpha, theta/beta)
#	•	Spectral entropy, median frequency

## 7. Thoracic and Abdominal Respiration

In [ ]:
# Load the MAT file
# Calculate  Mean, variance, RMS, peak-to-peak, zero crossings

In [ ]:
#Frequency-domain features (FFT, Welch PSD):
#Relative power in delta, theta, alpha, beta, gamma bands
#	•	Bandpower ratios (e.g., delta/alpha, theta/beta)
#	•	Spectral entropy, median frequency

## 8. Cross-signal features: